En este cuaderno recojo las funciones necesarias para poder hacer el web scraping realizado en este proyecto.

In [ ]:
import requests
from bs4 import BeautifulSoup

def obtener_espectaculos_atrapalo():
    """
    Esta función realiza una solicitud HTTP a la página de Atrapalo para obtener información sobre espectáculos en Madrid.
    Luego, utiliza la biblioteca BeautifulSoup para extraer los nombres de los espectáculos, sus URLs y los precios asociados.
    Los resultados se almacenan en tres listas separadas: espectaculos, urls_shows y precios.
    
    Returns:
        espectaculos (list): Lista de nombres de los espectáculos.
        urls_shows (list): Lista de URLs de los shows.
        precios (list): Lista de precios de los shows.
    """
    espectaculos = []
    urls_shows = []
    precios = []

    for i in range(1, 7):
        url = f'https://www.atrapalo.com/entradas/madrid/teatro-y-danza/monologos/p-{i}/'
        
        # Hacemos la request a la página
        res1 = requests.get(url)
        # Creamos la sopa
        sopa = BeautifulSoup(res1.content, 'html.parser')

        # Sacamos NOMBRES e iteramos para añadirlos a una lista
        nombres = sopa.find_all("h2", {"class": "clear nombre"})
        for i in nombres:
            espectaculos.append(i.getText().strip())
        
        # Sacamos urls de los shows e iteramos para añadirlos a una lista
        urls = sopa.find_all("div", {"data-url": True})
        for url in urls:
            urls_shows.append(url['data-url'])
        
        # Sacamos los precios de los shows e iteramos para añadirlos a una lista    
        p = sopa.find_all("span", {"class": "value"})
        for i in p:
            precios.append(i.getText().strip())
    
    return espectaculos, urls_shows, precios

In [ ]:
from selenium import webdriver
from time import sleep
import numpy as np

def obtener_info_espectaculos(urls_shows):
    """
    Esta función utiliza Selenium y el controlador de Chrome (webdriver.Chrome) para obtener información adicional
    sobre los espectáculos utilizando las URLs previamente obtenidas.
    
    Args:
        urls_shows (list): Lista de URLs de los espectáculos obtenidos desde la página de Atrapalo.
    
    Returns:
        monologos_madrid (dict): Diccionario que contiene la información adicional de los espectáculos.
            - "artista": Lista con el nombre del artista de cada espectáculo.
            - "lugar": Lista con el lugar del espectáculo.
            - "nota": Lista con la nota del espectáculo.
            - "n_opiniones": Lista con el número de opiniones del espectáculo.
            - "duracion": Lista con la duración del espectáculo.
    """
    monologos_madrid = {
        "artista": [],
        "lugar": [],
        "nota": [],
        "n_opiniones": [],
        "duracion": []
    }

    driver = webdriver.Chrome()

    for url in urls_shows:
        driver.get(f"https://www.atrapalo.com{url}") 
        driver.implicitly_wait(5)

        # Aceptamos las cookies de Atrapalo (si es necesario)
        try:
            driver.find_element("css selector", "#CybotCookiebotDialogBodyButtonAccept").click()
            sleep(2)
        except:
            pass

        # Capturamos el nombre del artista
        try:
            monologos_madrid["artista"].append(driver.find_element("xpath", '/html/body/div[4]/div[1]/section/section[1]/div/div[1]/div[3]/a').text)
        except:
            monologos_madrid["artista"].append(np.nan)

        # Capturamos el lugar
        try:
            monologos_madrid["lugar"].append(driver.find_element("xpath", '//*[@id="widgetMapa"]/ul/li[2]/a').text)
        except:
            monologos_madrid["lugar"].append(np.nan)

        # Capturamos la nota
        try:
            monologos_madrid["nota"].append(driver.find_element("css selector", '#ranking-item-wrapper_0 > div.opinions-ranking__header > div > div.overall__body > div > span.overall__rating').text)
        except:
            monologos_madrid["nota"].append(np.nan)

        # Capturamos el número de opiniones
        try:
            monologos_madrid["n_opiniones"].append(driver.find_element("css selector", '#ranking-item-wrapper_0 > div.opinions-ranking__header > div > div.overall__body > div > span.overall__opinions > span:nth-child(1)').text)
        except:
            monologos_madrid["n_opiniones"].append(np.nan)

        # Capturamos la duración
        try:
            monologos_madrid["duracion"].append(driver.find_element("xpath", '/html/body/div[4]/div[1]/section/section[2]/div[1]/ul/div[1]/li[2]/p').text)
        except:
            monologos_madrid["duracion"].append(np.nan)

    driver.quit()
    return monologos_madrid

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import numpy as np
from tqdm import tqdm

def obtener_info_locales_comedia(locales_lista):
    """
    Esta función utiliza Selenium y el controlador de Chrome (webdriver.Chrome) para obtener información adicional
    sobre los locales de comedia en Madrid utilizando la lista de locales previamente obtenida.
    
    Args:
        locales_lista (list): Lista de nombres de los locales de comedia en Madrid.
    
    Returns:
        locales_comedia_madrid (dict): Diccionario que contiene la información adicional de los locales de comedia.
            - "local": Lista con el nombre de cada local de comedia.
            - "direccion": Lista con la dirección del local.
            - "tipo_local": Lista con el tipo de local.
            - "valoracion": Lista con la valoración del local.
            - "numero_reseñas": Lista con el número de reseñas del local.
            - "enlace_web": Lista con el enlace web del local.
            - "puntuacion_entradas_com": Lista con la puntuación de entradas.com del local.
            - "numero_entradas_com": Lista con el número de puntuaciones de entradas.com del local.
            - "puntuacion_facebook": Lista con la puntuación de Facebook del local.
            - "numero_puntuacion_facebook": Lista con el número de puntuaciones de Facebook del local.
    """
    locales_comedia_madrid = {
        "local": [],
        "direccion": [],
        "tipo_local": [],
        "valoracion": [],
        "numero_reseñas": [],
        "enlace_web": [],
        "puntuacion_entradas_com": [],
        "numero_entradas_com": [],
        "puntuacion_facebook": [],
        "numero_puntuacion_facebook": []
    }

    driver = webdriver.Chrome()
    driver.get("https://www.google.com/")
    driver.implicitly_wait(5)

    driver.find_element("css selector", "#W0wltc > div").click()
    sleep(2)

    for local in tqdm(locales_lista):

        # Hacemos que busque el local y le dé a ENTER
        driver.find_element("xpath", '//*[@id="APjFqb"]').send_keys(local, Keys.ENTER)

        try:
            locales_comedia_madrid["direccion"].append(driver.find_element('css selector', 'span.LrzXr').text)
        except:
            locales_comedia_madrid["direccion"].append(np.nan)

        try:
            locales_comedia_madrid["tipo_local"].append(driver.find_element('css selector', 'div.zloOqf.kpS1Ac.vk_gy').text)
        except:
            locales_comedia_madrid["tipo_local"].append(np.nan)

        try:
            locales_comedia_madrid["valoracion"].append(driver.find_element("css selector", "span.Aq14fc").text)
        except:
            locales_comedia_madrid["valoracion"].append(np.nan)

        try:
            locales_comedia_madrid["numero_reseñas"].append(driver.find_element("css selector", "span.hqzQac a span").text)
        except:
            locales_comedia_madrid["numero_reseñas"].append(np.nan)

        try:
            web_element = driver.find_element('css selector', 'a.ab_button[href^="http://"], a.ab_button[href^="https://"]')
            locales_comedia_madrid["enlace_web"].append(web_element.get_attribute('href'))
        except:
            locales_comedia_madrid["enlace_web"].append(np.nan)

        try:
            locales_comedia_madrid["puntuacion_entradas_com"].append(driver.find_element('css selector', '.inaKse.G5rmf').text)
        except:
            locales_comedia_madrid["puntuacion_entradas_com"].append(np.nan)

        try:
            locales_comedia_madrid["numero_entradas_com"].append(driver.find_element('css selector', '.inaKse.KM6XSd').text)
        except:
            locales_comedia_madrid["numero_entradas_com"].append(np.nan)

        try:
            locales_comedia_madrid["puntuacion_facebook"].append(driver.find_element('css selector', 'a:nth-child(2) > .inaKse.G5rmf').text)
        except:
            locales_comedia_madrid["puntuacion_facebook"].append(np.nan)

        try:
            locales_comedia_madrid["numero_puntuacion_facebook"].append(driver.find_element('css selector', 'a:nth-child(2) > .inaKse.KM6XSd').text)
        except:
            locales_comedia_madrid["numero_puntuacion_facebook"].append(np.nan)

        driver.find_element("css selector", "#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg").click()
        sleep(2)

    driver.quit()
    return locales_comedia_madrid

# Unica llamada

Para poder hacer la extracción de todos los datos de una, podemos usar esta función:

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import numpy as np
from tqdm import tqdm

def obtener_info_espectaculos_locales():
    """
    Esta función obtiene información sobre espectáculos y locales de comedia en Madrid utilizando las páginas de Atrapalo y Google.
    
    Returns:
        info_total (dict): Diccionario con la información combinada de los espectáculos y los locales de comedia.
            - "espectaculos": Diccionario con información adicional de los espectáculos.
                - "nombres": Lista con los nombres de los espectáculos.
                - "urls_shows": Lista con las URLs de los espectáculos.
                - "precios": Lista con los precios de los espectáculos.
            - "locales_comedia": Diccionario con información adicional de los locales de comedia.
                - "local": Lista con el nombre de cada local de comedia.
                - "direccion": Lista con la dirección del local.
                - "tipo_local": Lista con el tipo de local.
                - "valoracion": Lista con la valoración del local.
                - "numero_reseñas": Lista con el número de reseñas del local.
                - "enlace_web": Lista con el enlace web del local.
                - "puntuacion_entradas_com": Lista con la puntuación de entradas.com del local.
                - "numero_entradas_com": Lista con el número de puntuaciones de entradas.com del local.
                - "puntuacion_facebook": Lista con la puntuación de Facebook del local.
                - "numero_puntuacion_facebook": Lista con el número de puntuaciones de Facebook del local.
    """
    # Parte 1: Obtener información de los espectáculos desde Atrapalo
    espectaculos = []
    urls_shows = []
    precios = []

    for i in range(1, 7):
        url = f'https://www.atrapalo.com/entradas/madrid/teatro-y-danza/monologos/p-{i}/'
        
        # Hacemos la request a la página
        res1 = requests.get(url)
        # Creamos la sopa
        sopa = BeautifulSoup(res1.content, 'html.parser')

        # Sacamos NOMBRES e iteramos para añadirlos a una lista
        nombres = sopa.find_all("h2", {"class": "clear nombre"})
        for i in nombres:
            espectaculos.append(i.getText().strip())
        
        # Sacamos urls de los shows e iteramos para añadirlos a una lista
        urls = sopa.find_all("div", {"data-url": True})
        for url in urls:
            urls_shows.append(url['data-url'])
        
        # Sacamos los precios de los shows e iteramos para añadirlos a una lista    
        p = sopa.find_all("span", {"class": "value"})
        for i in p:
            precios.append(i.getText().strip())

    # Parte 2: Obtener información de los locales de comedia desde Google
    locales_comedia_madrid = {
        "local": [],
        "direccion": [],
        "tipo_local": [],
        "valoracion": [],
        "numero_reseñas": [],
        "enlace_web": [],
        "puntuacion_entradas_com": [],
        "numero_entradas_com": [],
        "puntuacion_facebook": [],
        "numero_puntuacion_facebook": []
    }

    driver = webdriver.Chrome()
    driver.get("https://www.google.com/")
    driver.implicitly_wait(5)

    driver.find_element("css_selector", "#W0wltc > div").click()
    sleep(2)

    for local in tqdm(locales_lista):
        # Hacemos que busque el local y le dé a ENTER
        driver.find_element("xpath", '//*[@id="APjFqb"]').send_keys(local, Keys.ENTER)

        try:
            locales_comedia_madrid["direccion"].append(driver.find_element('css_selector', 'span.LrzXr').text)
        except:
            locales_comedia_madrid["direccion"].append(np.nan)

        try:
            locales_comedia_madrid["tipo_local"].append(driver.find_element('css_selector', 'div.zloOqf.kpS1Ac.vk_gy').text)
        except:
            locales_comedia_madrid["tipo_local"].append(np.nan)

        try:
            locales_comedia_madrid["valoracion"].append(driver.find_element("css_selector", "span.Aq14fc").text)
        except:
            locales_comedia_madrid["valoracion"].append(np.nan)

        try:
            locales_comedia_madrid["numero_reseñas"].append(driver.find_element("css_selector", "span.hqzQac a span").text)
        except:
            locales_comedia_madrid["numero_reseñas"].append(np.nan)

        try:
            web_element = driver.find_element('css_selector', 'a.ab_button[href^="http://"], a.ab_button[href^="https://"]')
            locales_comedia_madrid["enlace_web"].append(web_element.get_attribute('href'))
        except:
            locales_comedia_madrid["enlace_web"].append(np.nan)

        try:
            locales_comedia_madrid["puntuacion_entradas_com"].append(driver.find_element('css_selector', '.inaKse.G5rmf').text)
        except:
            locales_comedia_madrid["puntuacion_entradas_com"].append(np.nan)

        try:
            locales_comedia_madrid["numero_entradas_com"].append(driver.find_element('css_selector', '.inaKse.KM6XSd').text)
        except:
            locales_comedia_madrid["numero_entradas_com"].append(np.nan)

        try:
            locales_comedia_madrid["puntuacion_facebook"].append(driver.find_element('css_selector', 'a:nth-child(2) > .inaKse.G5rmf').text)
        except:
            locales_comedia_madrid["puntuacion_facebook"].append(np.nan)

        try:
            locales_comedia_madrid["numero_puntuacion_facebook"].append(driver.find_element('css_selector', 'a:nth-child(2) > .inaKse.KM6XSd').text)
        except:
            locales_comedia_madrid["numero_puntuacion_facebook"].append(np.nan)

        driver.find_element("css_selector", "#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg").click()
        sleep(2)

    driver.quit()

    # Crear el diccionario final con toda la información combinada